In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
Gemini_api_key = os.environ["GEMINI_API_KEY"]

In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain import hub

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import AstraDB

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
GoogleGeminiEmbeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=Gemini_api_key )

In [8]:
loader = PyPDFLoader("./Brown_Churchill_Complex_Variables_and_Ap.pdf")
pages = loader.load_and_split()

In [9]:
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1200, chunk_overlap = 400)
splits = text_splitter.split_documents(pages)

In [11]:
vstore = AstraDB(
    embedding=GoogleGeminiEmbeddings,
    collection_name="Complex_variables",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [12]:
ids = vstore.add_documents(splits)

In [13]:
retriever = vstore.as_retriever()
template = """
context: {context}

question: {question}
"""
prompt = PromptTemplate.from_template(template)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.0, google_api_key=Gemini_api_key, )

In [14]:
genai_llm = GoogleGenerativeAI(model = "gemini-pro", temperature=0.0, google_api_key=Gemini_api_key)

In [15]:
def pages_adjustments(pages):
    return "\n\n".join(page.page_content for page in pages)

In [16]:
context_chain = retriever | pages_adjustments

In [17]:
main_chain = (
    {"context" : context_chain, "question": RunnablePassthrough()}
    |prompt
    | genai_llm
    | StrOutputParser()
)

In [76]:
main_chain.invoke("What is Cauchy Reimann Equation? Also Proof the equations.")

"The Cauchy-Riemann equations are a system of two partial differential equations that are necessary and sufficient conditions for a complex function to be holomorphic (also known as analytic). They are named after Augustin-Louis Cauchy and Bernhard Riemann, who independently discovered them in the 19th century.\n\nThe Cauchy-Riemann equations are:\n\n```\n∂u/∂x = ∂v/∂y\n∂u/∂y = -∂v/∂x\n```\n\nwhere u and v are the real and imaginary parts of the complex function f(z), respectively.\n\nTo prove the Cauchy-Riemann equations, we can use the definition of the derivative of a complex function:\n\n```\nf'(z) = lim(h->0) [f(z+h) - f(z)]/h\n```\n\nIf f(z) is holomorphic, then its derivative exists at every point in its domain. We can use this fact to compute the partial derivatives of u and v:\n\n```\n∂u/∂x = lim(h->0) [u(x+h, y) - u(x, y)]/h\n= lim(h->0) [Re(f(z+h)) - Re(f(z))]/h\n= lim(h->0) [Re(f(z+h) - f(z))]/h\n= Re(f'(z))\n```\n\nSimilarly, we can compute the other partial derivatives:\n

In [47]:
from langchain_core.output_parsers import JsonOutputParser

In [50]:
check_prompt = PromptTemplate.from_template("""
If the given statement is just a task then pick the keywords and make a questions of them to know them and then add the task also after a dot. 
statement: {statement}
""")
task_check_chain = (
    {"statement": RunnablePassthrough()}
    | check_prompt
    | genai_llm
    | StrOutputParser()
)

In [49]:
task_check_chain.invoke("Prove Cauchy Reimann Equation.")

OutputParserException: Invalid json output: Keywords:
- Cauchy Reimann Equation
- Complex Function
- Partial Derivatives
- Analytic Function

Questions:
- What is the Cauchy Reimann Equation?
- How is the Cauchy Reimann Equation used to define an analytic function?
- What are the partial derivatives of a complex function?
- How are the partial derivatives of a complex function related to the Cauchy Reimann Equation?

Task:
Prove the Cauchy Reimann Equation for a given complex function.

In [44]:
main_chain.invoke("Keywords:\n- Cauchy Reimann Equation\n- Complex Function\n- Partial Derivatives\n- Analytic Function\n\nQuestions:\n- What is the Cauchy Reimann Equation?\n- How is the Cauchy Reimann Equation used to define an analytic function?\n- What are the partial derivatives of a complex function?\n- How are the partial derivatives of a complex function related to the Cauchy Reimann Equation?\n\nTask:\nProve the Cauchy Reimann Equation for a given complex function.'")

"Keywords:\n- Cauchy Reimann Equation\n- Complex Function\n- Partial Derivatives\n- Analytic Function\n\nQuestions:\n- What is the Cauchy Reimann Equation?\n- How is the Cauchy Reimann Equation used to define an analytic function?\n- What are the partial derivatives of a complex function?\n- How are the partial derivatives of a complex function related to the Cauchy Reimann Equation?\n\nTask:\nProve the Cauchy Reimann Equation for a given complex function.\n\nAnswers:\n- The Cauchy Reimann Equation is a system of two partial differential equations that relate the partial derivatives of the real and imaginary parts of a complex function.\n- The Cauchy Reimann Equation is used to define an analytic function. A complex function is analytic at a point if it satisfies the Cauchy Reimann Equation at that point.\n- The partial derivatives of a complex function are the derivatives of the real and imaginary parts of the function with respect to the real and imaginary parts of the independent va

In [45]:
print("Keywords:\n- Cauchy Reimann Equation\n- Complex Function\n- Partial Derivatives\n- Analytic Function\n\nQuestions:\n- What is the Cauchy Reimann Equation?\n- How is the Cauchy Reimann Equation used to define an analytic function?\n- What are the partial derivatives of a complex function?\n- How are the partial derivatives of a complex function related to the Cauchy Reimann Equation?\n\nTask:\nProve the Cauchy Reimann Equation for a given complex function.\n\nAnswers:\n- The Cauchy Reimann Equation is a system of two partial differential equations that relate the partial derivatives of the real and imaginary parts of a complex function.\n- The Cauchy Reimann Equation is used to define an analytic function. A complex function is analytic at a point if it satisfies the Cauchy Reimann Equation at that point.\n- The partial derivatives of a complex function are the derivatives of the real and imaginary parts of the function with respect to the real and imaginary parts of the independent variable.\n- The partial derivatives of a complex function are related to the Cauchy Reimann Equation by the following equations:\n\n```\nu_x = v_y\nu_y = -v_x\n```\n\nwhere u and v are the real and imaginary parts of the complex function, respectively.\n\nProof of the Cauchy Reimann Equation:\n\nLet f(z) be a complex function defined by\n\n```\nf(z) = u(x, y) + iv(x, y)\n```\n\nwhere u and v are real-valued functions of the real variables x and y.\n\nThe partial derivatives of f(z) are given by\n\n```\nf'(z) = u_x(x, y) + iv_x(x, y)\n```\n\nand\n\n```\nf'(z) = u_y(x, y) + iv_y(x, y)\n```\n\nIf f(z) is analytic at a point z0, then f'(z0) exists. This means that the partial derivatives of u and v exist at (x0, y0), where z0 = x0 + iy0.\n\nThe Cauchy Reimann Equation is a system of two partial differential equations that relate the partial derivatives of u and v. These equations are\n\n```\nu_x = v_y\nu_y = -v_x\n```\n\nTo prove the Cauchy Reimann Equation, we need to show that these equations hold at (x0, y0).\n\nDifferentiating the first equation with respect to y and the second equation with respect to x, we get\n\n```\nu_{xy} = v_{yy}\nu_{yx} = -v_{xy}\n```\n\nSince the mixed partial derivatives of u and v are equal, we have\n\n```\nu_{xy} = u_{yx}\n```\n\nThis implies that\n\n```\nv_{yy} = -v_{xy}\n```\n\nIntegrating both sides of this equation with respect to y, we get\n\n```\nv_y = -v_x + g(x)\n```\n\nwhere g(x) is a function of x.\n\nSubstituting this expression for v_y into the first equation of the Cauchy Reimann Equation, we get\n\n```\nu_x = -v_x + g(x)\n```\n\nIntegrating both sides of this equation with respect to x, we get\n\n```\nu = -v + g(x)y + h(y)\n```\n\nwhere h(y) is a function of y.\n\nSubstituting this expression for u into the second equation of the Cauchy Reimann Equation, we get\n\n```\n-v_x + g(x)y + h'(y) = -v_x\n```\n\nThis implies that\n\n```\ng(x)y + h'(y) = 0\n```\n\nSince this equation must hold for all values of x and y, we must have\n\n```\ng(x) = 0\n```\n\nand\n\n```\nh'(y) = 0\n```\n\nThis implies that\n\n```\nu = -v + C\n```\n\nwhere C is a constant.\n\nTherefore, the Cauchy Reimann Equation holds at (x0, y0), and hence f(z) is analytic at z0.")

Keywords:
- Cauchy Reimann Equation
- Complex Function
- Partial Derivatives
- Analytic Function

Questions:
- What is the Cauchy Reimann Equation?
- How is the Cauchy Reimann Equation used to define an analytic function?
- What are the partial derivatives of a complex function?
- How are the partial derivatives of a complex function related to the Cauchy Reimann Equation?

Task:
Prove the Cauchy Reimann Equation for a given complex function.

Answers:
- The Cauchy Reimann Equation is a system of two partial differential equations that relate the partial derivatives of the real and imaginary parts of a complex function.
- The Cauchy Reimann Equation is used to define an analytic function. A complex function is analytic at a point if it satisfies the Cauchy Reimann Equation at that point.
- The partial derivatives of a complex function are the derivatives of the real and imaginary parts of the function with respect to the real and imaginary parts of the independent variable.
- The parti